Нахождение контекстов с авторскими номинациями

In [4]:
import pandas as pd
import re
import os

In [5]:
# Пути к файлам
corpus_file = r"C:\Users\Sony\OneDrive\Рабочий стол\Переподготовка ВШЭ\проект\2\LOTR\15.02 corp\corp_nom.xlsx"
names_file = r"C:\Users\Sony\OneDrive\Рабочий стол\Переподготовка ВШЭ\проект\2\LOTR\nom.xlsx"
output_file = r"C:\Users\Sony\OneDrive\Рабочий стол\Переподготовка ВШЭ\проект\2\LOTR\15.02 corp\corp_nom_columns.xlsx"

In [6]:
# Загружаем корпус с параллельными текстами
df_corpus = pd.read_excel(corpus_file)

# Загружаем файл с именами
df_names = pd.read_excel(names_file)

print(f"Корпус: {len(df_corpus)} строк")
print(f"Имена: {len(df_names)} строк")

Корпус: 4325 строк
Имена: 1596 строк


In [7]:
# Функция для очистки имени
def clean_name(name):
    if pd.isna(name):
        return ""
    name = str(name).strip().lower()
    name = re.sub(r'\(.*?\)', '', name)  # Удаляем скобки с содержимым
    name = re.sub(r'["\']', '', name)  # Удаляем кавычки
    name = re.sub(r'\s+', ' ', name)  # Удаляем лишние пробелы
    return name.strip()

# Собираем имена из каждого столбца
english_names = []
russian_1_names = []
russian_2_names = []
russian_3_names = []

# Определяем, какие столбцы есть в файле с именами
cols = list(df_names.columns)

if len(cols) >= 1:
    english_names = [clean_name(name) for name in df_names[cols[0]].dropna() if clean_name(name)]
if len(cols) >= 2:
    russian_1_names = [clean_name(name) for name in df_names[cols[1]].dropna() if clean_name(name)]
if len(cols) >= 3:
    russian_2_names = [clean_name(name) for name in df_names[cols[2]].dropna() if clean_name(name)]
if len(cols) >= 4:
    russian_3_names = [clean_name(name) for name in df_names[cols[3]].dropna() if clean_name(name)]

# Убираем дубликаты и сортируем от длинных к коротким
english_names = sorted(set(english_names), key=len, reverse=True)
russian_1_names = sorted(set(russian_1_names), key=len, reverse=True)
russian_2_names = sorted(set(russian_2_names), key=len, reverse=True)
russian_3_names = sorted(set(russian_3_names), key=len, reverse=True)

print(f"Найдено оригинальных имен: {len(english_names)}")
print(f"Найдено имен в переводе 1: {len(russian_1_names)}")
print(f"Найдено имен в переводе 2: {len(russian_2_names)}")
print(f"Найдено имен в переводе 3: {len(russian_3_names)}")

Найдено оригинальных имен: 1556
Найдено имен в переводе 1: 1048
Найдено имен в переводе 2: 835
Найдено имен в переводе 3: 1474


In [8]:
# Функция для поиска имен в тексте
def find_names(text, names_list):
    if pd.isna(text):
        return ""
    
    text_lower = str(text).lower()
    found = []
    
    for name in names_list:
        if not name:
            continue
            
        # Создаем регулярное выражение для поиска имени как отдельного слова
        pattern = r'\b' + re.escape(name) + r'\b'
        
        if re.search(pattern, text_lower):
            found.append(name)
    
    return ", ".join(found)

In [9]:
# Ищем имена в каждом столбце 
# Копируем исходный датафрейм
result_df = df_corpus.copy()

# Проверяем, какие столбцы есть в корпусе
corpus_cols = list(df_corpus.columns)

# Ищем имена в английском столбце
if 'english' in corpus_cols:
    print("Ищу имена в английском тексте...")
    result_df['nom.orig'] = result_df['english'].apply(lambda x: find_names(x, english_names))
    
    # Статистика
    filled = result_df['nom.orig'].str.len() > 0
    print(f"Найдено имен в {filled.sum()} строках ({filled.sum()/len(result_df)*100:.1f}%)")

# Ищем имена в русских столбцах
for i in range(1, 4):
    col_name = f'russian_{i}'
    if col_name in corpus_cols:
        # Выбираем соответствующий список имен
        if i == 1:
            names_to_use = russian_1_names
        elif i == 2:
            names_to_use = russian_2_names
        else:
            names_to_use = russian_3_names
        
        print(f"Ищу имена в {col_name}...")
        result_df[f'nom.{i}'] = result_df[col_name].apply(lambda x: find_names(x, names_to_use))
        
        # Статистика
        filled = result_df[f'nom.{i}'].str.len() > 0
        print(f"Найдено имен в {filled.sum()} строках ({filled.sum()/len(result_df)*100:.1f}%)")

Ищу имена в английском тексте...
Найдено имен в 4023 строках (93.0%)
Ищу имена в russian_1...
Найдено имен в 2636 строках (60.9%)
Ищу имена в russian_2...
Найдено имен в 2398 строках (55.4%)
Ищу имена в russian_3...
Найдено имен в 2665 строках (61.6%)


In [10]:
# Сохраняем в Excel
result_df.to_excel(output_file, index=False)
print(f"\nРезультат сохранен в: {output_file}")

# Показываем результ
print(result_df.head())


Результат сохранен в: C:\Users\Sony\OneDrive\Рабочий стол\Переподготовка ВШЭ\проект\2\LOTR\15.02 corp\corp_nom_columns.xlsx
                                             english  \
0  A bell clanged; and from the Watchers there we...   
1  A boding of ill we thought it, my father and I...   
2  A broad stairway climbed from the Deep up to t...   
3  A cliff frowned upon their right; to their lef...   
4  A deep boom rumbled through the City like thun...   

                                           russian_1  \
0          Ударил колокол, и дико взвыли Соглядатаи.   
1  Дурным предзнаменованьем сочли это мы с отцом,...   
2  Широкая лестница вела от ущелья на крепостную ...   
3  Справа нависали скалы; слева высился серый скл...   
4  Казалось, гром из поднебесья раскатился по гор...   

                                           russian_2  \
0  Зазвонил колокол, и Стражи разразились пронзит...   
1  Мы сочли это дурным знаком, ибо никто ничего н...   
2  Арагорн пересек внутренний дво